In [1]:
import cv2
import numpy as np


In [2]:
# we first start captyring video. somple code and we exit using the esc key.
# second start object tracking using meanshift first and provide the haarcascade of frontal face features.
# determine the area where blur is to be applied. store it in variable and provide that variable to blur function
# next the area determined for blur is given the result of blur for application
# display the frame

In [3]:
# def tracking(frame):
#     cap = cv2.VideoCapture(0)

#     ret, incoming_frame = cap.read()

#     # classify facial features which are needed
#     face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#     # 
#     face_rects = face_cascade.detectMultiScale(frame)

#     # from face_rects extract the face 
#     (face_x, face_y, w, h) = tuple(face_rects[0])
#     track_window = (face_x, face_y, w, h)

#     # region of interest is the whole face rectangle. from (0,0) at top of face till end of face which is detected according to cascade 
#     # classifier method
#     roi = frame[face_y: face_y + h, face_x: face_x + w]
#     # convert roi to hsv for better color detection. because hsv allows for that
#     hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
#     # create histogram of roi as is explained in docs of meanshift. because it is required . 
#     roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])
#     # normalize for all frames
#     cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
#     term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1) # search
#     return roi_hist, term_crit, track_window



In [2]:
# select default webcam as input
cap = cv2.VideoCapture(0)

# # if cameras is detected, start reading from the cam
# ret, incoming_frame = cap.read()

# classify facial features which are needed
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')




while True:

    ret, incoming_frame = cap.read()
    # 
    face_rects = face_cascade.detectMultiScale(incoming_frame)

    # from face_rects extract the face 
    (face_x, face_y, w, h) = tuple(face_rects[0])
    track_window = (face_x, face_y, w, h)

    # region of interest is the whole face rectangle. from (0,0) at top of face till end of face which is detected according to cascade 
    # classifier method
    roi = incoming_frame[face_y: face_y + h, face_x: face_x + w]

    # convert roi to hsv for better color detection. because hsv allows for that
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    #       create histogram of roi as is explained in docs of meanshift. because it is required . 
    roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])

    # normalize for all frames
    cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

    term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1) # search
    
    if ret == True:

    
        hsv = cv2.cvtColor(incoming_frame, cv2.COLOR_BGR2HSV)

        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1) # search
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        x,y,w,h = track_window
        cv2.rectangle(incoming_frame, (x,y), (x+w,y+h), (0,255,0), 5)
        # determine area for blur
        blur_area = incoming_frame[y:y+h, x:x+w]
        # apply blur
        blur = cv2.GaussianBlur(blur_area, (51,51), 0)
        # provide the blur to area to be blurred
        incoming_frame[y:y+h, x:x+w] = blur

        # start display the input frames in window named "Face Blur"
        cv2.imshow('Face Blur', incoming_frame)

        # assign the esc key as exit button and also wait 10ms before exiting
        k = cv2.waitKey(10) & 0xFF
        # if esc was pressed exit from loop. i.e. close capture
        if k == 27:
            break
    else:
        break


cap.release()
cv2.destroyAllWindows()